In [ ]:
import tifffile
import zarr

In [ ]:
z = zarr.open('/mnt/raid_hdd/combined_uv_volume.zarr', mode='r')
print(z.tree())


In [31]:
import zarr
import tifffile
import os

# Open the Zarr array
raw = zarr.open("/home/sean/Desktop/s1_segments/3336/predictions.zarr", mode='r')
ink = raw['ink_final']

# Get the shape of the array
z_layers, height, width = ink.shape  # Assuming shape is (Z, Y, X)

# Create output directory if it doesn't exist
output_dir = "/home/sean/Desktop/s1_segments/3336/ink_layers"
os.makedirs(output_dir, exist_ok=True)

# Iterate through Z layers
for z in range(z_layers):
    # Read the z-layer
    layer = ink[z, :, :]

    # Create output filename
    output_path = os.path.join(output_dir, f"{z:04d}.tif")

    # Save as TIFF
    tifffile.imwrite(output_path, layer)
    print(f"Saved layer {z} to {output_path}")

Saved layer 0 to /home/sean/Desktop/s1_segments/3336/ink_layers/0000.tif
Saved layer 1 to /home/sean/Desktop/s1_segments/3336/ink_layers/0001.tif
Saved layer 2 to /home/sean/Desktop/s1_segments/3336/ink_layers/0002.tif
Saved layer 3 to /home/sean/Desktop/s1_segments/3336/ink_layers/0003.tif
Saved layer 4 to /home/sean/Desktop/s1_segments/3336/ink_layers/0004.tif
Saved layer 5 to /home/sean/Desktop/s1_segments/3336/ink_layers/0005.tif
Saved layer 6 to /home/sean/Desktop/s1_segments/3336/ink_layers/0006.tif
Saved layer 7 to /home/sean/Desktop/s1_segments/3336/ink_layers/0007.tif
Saved layer 8 to /home/sean/Desktop/s1_segments/3336/ink_layers/0008.tif
Saved layer 9 to /home/sean/Desktop/s1_segments/3336/ink_layers/0009.tif
Saved layer 10 to /home/sean/Desktop/s1_segments/3336/ink_layers/0010.tif
Saved layer 11 to /home/sean/Desktop/s1_segments/3336/ink_layers/0011.tif
Saved layer 12 to /home/sean/Desktop/s1_segments/3336/ink_layers/0012.tif
Saved layer 13 to /home/sean/Desktop/s1_segments

In [4]:
import shutil
import os

def copy_and_number(source_image, start_number, num_copies):
    # Check if source file exists
    if not os.path.exists(source_image):
        print(f"Error: Source file '{source_image}' not found")
        return

    # Get the directory and file extension
    directory = os.path.dirname(source_image) or '.'
    _, ext = os.path.splitext(source_image)

    # Create numbered copies
    for i in range(num_copies):
        new_number = start_number + i
        new_filename = f"{new_number}{ext}"
        new_path = os.path.join(directory, new_filename)

        try:
            shutil.copy2(source_image, new_path)
            print(f"Created: {new_filename}")
        except Exception as e:
            print(f"Error creating {new_filename}: {e}")

# Example usage
if __name__ == "__main__":
    source_image = "/home/sean/Desktop/s1_segments/5753/20230702185753_inklabels.png"
    start_number = 1
    num_copies =20

    copy_and_number(source_image, start_number, num_copies)

Created: 1.png
Created: 2.png
Created: 3.png
Created: 4.png
Created: 5.png
Created: 6.png
Created: 7.png
Created: 8.png
Created: 9.png
Created: 10.png
Created: 11.png
Created: 12.png
Created: 13.png
Created: 14.png
Created: 15.png
Created: 16.png
Created: 17.png
Created: 18.png
Created: 19.png
Created: 20.png


In [1]:
import os
import glob
import re
from tqdm import tqdm

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'([0-9]+)', s)]

def check_layers_in_range(parent_folder, start=25, stop=35):
    """
    Go through each segment folder in `parent_folder` (ones that have a 'layers' subdirectory)
    and check if the slice indices [start..stop] are valid based on the number of layer files.

    If any slice index is out of range, we print a warning message.
    """

    # 1) Find all subdirectories that contain a 'layers' folder
    segment_folders = [
        f.path for f in os.scandir(parent_folder)
        if f.is_dir() and os.path.exists(os.path.join(f.path, 'layers'))
    ]

    if not segment_folders:
        raise ValueError(f"No valid segment folders found in {parent_folder}")

    print(f"Found {len(segment_folders)} segment folders to check.")

    # 2) For each segment, gather layer files and sort them the same way
    for folder in tqdm(segment_folders, desc="Checking segments"):
        seg_name = os.path.basename(folder)
        layers_dir = os.path.join(folder, 'layers')

        # Collect and sort .tif layer files
        layers = glob.glob(os.path.join(layers_dir, '*.tif'))
        layers.sort(key=natural_sort_key)

        if not layers:
            print(f"WARNING: No .tif files found in {layers_dir}")
            continue

        # 3) Determine how many slices we plan to read
        num_slices = stop - start + 1

        # 4) Check if each requested slice index is within range
        #    of the sorted `layers` list
        out_of_range = False
        for idx in range(num_slices):
            file_index = start + idx
            if file_index >= len(layers):
                # This means we'd "fall back" to the last layer file in your original script
                print(
                    f"Out-of-range: Segment '{seg_name}' "
                    f"needs slice index {file_index} but only has {len(layers)} files."
                )
                out_of_range = True
                # We don't break here because we may want to see *all* out-of-range indices

        # 5) Optional: If you only want to print a single message for each segment
        # that has any missing slices, you could do:
        if out_of_range:
            print(f" -> Segment '{seg_name}' does NOT have all slices for [{start}..{stop}].")
        else:
            print(f" -> Segment '{seg_name}' is good (has at least {stop+1} files).")

if __name__ == "__main__":
    parent_folder = "/home/sean/Desktop/s1_segments/"
    check_layers_in_range(parent_folder, start=25, stop=35)


Found 8 segment folders to check.


Checking segments: 100%|██████████| 8/8 [00:00<00:00, 2137.36it/s]

Out-of-range: Segment '4423' needs slice index 25 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 26 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 27 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 28 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 29 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 30 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 31 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 32 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 33 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 34 but only has 21 files.
Out-of-range: Segment '4423' needs slice index 35 but only has 21 files.
 -> Segment '4423' does NOT have all slices for [25..35].
Out-of-range: Segment '0901' needs slice index 25 but only has 21 files.
Out-of-range: Segment '0901' needs slice index 26 but only has 21 

In [ ]:
def compute_3dunet_feature_map_shapes(
    patch_size: tuple,
    f_maps: list,
    pool_kernel_size: int = 2
):
    """
    Compute approximate (channels, D, H, W) for each encoder and decoder level
    in a typical 3D U-Net with pool/upsample factor = pool_kernel_size.

    Parameters
    ----------
    patch_size : tuple
        (D, H, W) of the input volume.
    f_maps : list of int
        e.g. [32, 64, 128, 256, 320, 528] (one entry per U-Net level).
    pool_kernel_size : int
        Factor by which we downscale (encoder) or upscale (decoder) in each level.

    Returns
    -------
    enc_shapes : list of tuples
        List of (channels, D, H, W) from each encoder level (top to bottom).
    dec_shapes : list of tuples
        List of (channels, D, H, W) from each decoder level (bottom to top).

    Note
    ----
    We assume each encoder level is "before" the downsampling, and each decoder
    level is "after" the upsampling. This matches a fairly standard 3D U-Net design,
    but your exact code may differ slightly.
    """
    D, H, W = patch_size

    # ----- Encoder shapes -----
    # For each f_maps[i], assume we see the shape before pooling to the next stage.
    enc_shapes = []
    currentD, currentH, currentW = D, H, W
    for c in f_maps:
        enc_shapes.append((c, currentD, currentH, currentW))
        # after this level, we do a pooling of factor=2 in D,H,W
        currentD //= pool_kernel_size
        currentH //= pool_kernel_size
        currentW //= pool_kernel_size

    # ----- Decoder shapes -----
    # We go back up from deepest to shallower. Usually we do len(f_maps)-1 upsampling stages.
    dec_shapes = []
    for i in reversed(range(len(f_maps) - 1)):
        # upsample from the current shape by factor=2
        currentD *= pool_kernel_size
        currentH *= pool_kernel_size
        currentW *= pool_kernel_size
        dec_shapes.append((f_maps[i], currentD, currentH, currentW))

    return enc_shapes, dec_shapes


In [2]:
def estimate_3d_unet_vram(
    num_params: int,
    patch_size: tuple,
    dtype_size: int,
    batch_size: int,
    in_channels: int,
    out_channels: int,
    feature_map_sizes: list
):
    """
    Rough VRAM estimate for a 3D U-Net-like model.

    Parameters:
    -----------
    num_params : int
        Total number of trainable parameters in the model.
    patch_size : tuple of int (D, H, W)
        The size of the 3D input patch.
    dtype_size : int
        Bytes per floating-point value (2 for float16, 4 for float32, etc.).
    batch_size : int
        Batch size.
    in_channels : int
        Number of input channels.
    out_channels : int
        Number of output channels.
    feature_map_sizes : list of tuples
        Each tuple is (num_channels, D_level, H_level, W_level) for each layer.

    Returns:
    --------
    float
        Rough estimate of memory in bytes.
    """

    # 1) Parameter memory
    param_mem = num_params * dtype_size

    # 2) Input volume memory
    D, H, W = patch_size
    input_mem = batch_size * in_channels * D * H * W * dtype_size

    # 3) Output volume memory
    output_mem = batch_size * out_channels * D * H * W * dtype_size

    # 4) Intermediate feature maps memory
    fm_mem = 0
    for (c, d_level, h_level, w_level) in feature_map_sizes:
        fm_mem += batch_size * c * d_level * h_level * w_level * dtype_size

    # Multiply by 2 for forward + backward
    fm_mem *= 2

    # Total
    total_mem = param_mem + input_mem + output_mem + fm_mem
    return total_mem


# Example usage:
if __name__ == "__main__":
    # Suppose:
    num_params = 45074348         # 3 million parameters
    patch_size = (64, 192, 192)       # typical 3D patch
    dtype_size = 4                  # float32
    batch_size = 4
    in_channels = 1
    out_channels = 4

    # For demonstration: let's pretend we have 4 encoder levels and 4 decoder levels
    # Just made-up shapes and channel counts.
    feature_map_sizes = [
        (16, 64, 64, 64),   # encoder level 1
        (32, 32, 32, 32),   # encoder level 2
        (64, 16, 16, 16),   # encoder level 3
        (128, 8, 8, 8),     # encoder level 4
        (64, 16, 16, 16),   # decoder level 1
        (32, 32, 32, 32),   # decoder level 2
        (16, 64, 64, 64),   # decoder level 3
        (8,  64, 64, 64),   # final, etc. (just an example)
    ]

    rough_vram_bytes = estimate_3d_unet_vram(
        num_params,
        patch_size,
        dtype_size,
        batch_size,
        in_channels,
        out_channels,
        feature_map_sizes
    )

    print(f"Rough VRAM estimate: {rough_vram_bytes / 1e9:.2f} GB")


Rough VRAM estimate: 0.79 GB
